# Graph Construction: Network X

In this Notebook, we construct the graph of the Rt's Users during the Paro Nacional. Each graph corresponds to the Rt Network in moving windows of three days between April 28 of 2021 and June 30 of 2021

In [1]:
import pickle
from glob import glob
import numpy as np
import pandas as pd
import networkx as nx
from datetime import datetime
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix

In [3]:
with open('/mnt/disk2/Data/Pickle/user_indices.pkl','rb') as file:
    user_indices = pickle.load(file)

with open('/mnt/disk2/Data/Pickle/user_to_party_paro.pkl','rb') as file:
    user_to_party_paro = pickle.load(file)

In [4]:
# from user_indices we transform the keys as values and the values as keys
user_indices_2 = {value: key for key, value in user_indices.items()}

In [7]:
# Dates of the Paro Nacional
v1_start = '2021-04-28 00:00:00'
v1_end = '2021-06-27 00:00:00'
date1 = pd.date_range(start = v1_start, end = v1_end, freq = 'D')

v2_start = '2021-04-30 23:59:59'
v2_end = '2021-06-29 23:59:59'
date2 = pd.date_range(start = v2_start, end = v2_end, freq = 'D')
datestr = list(date2.strftime("%d-%m"))

After declaring our constructor function, we now proceed to create the loop over the adjcency matrices to create the graphs

Because Graph_tool doesn't have permissions to access the directories and creeate the files in the folder, run this line of code in the Linus Terminal to run the This code Below.

```
sudo ~/.conda/envs/gt/bin/python "/mnt/disk2/fcastrillon/Analysis-of-Tweets-During-the-2021-Social-Unrest/Code/utils/Graph Tool Saves.py"
```

In [ ]:
k = 1
files = glob('/mnt/disk2/Data/Matrices/*.npz')
for file in tqdm(files):
    
    # First, we load the stored info into a normal CSR matrix.
    data = np.load(file)
    indices = data['indices']
    indptr = data['indptr']
    shape = data['shape']
    data = data['data']
    A = csr_matrix((data, indices, indptr), shape = shape)
    
    # Now we can create the graph using 'graph_tool'.
    idx = A.nonzero()
    weights = A[idx]
    g = gt.Graph(directed = False)
    g.add_edge_list(np.transpose(idx))
    ew = g.new_edge_property("double")
    ew.a = weights 
    g.ep['edge_weight'] = ew

    date = date_list[k] # This gets the end of day date of every graph to store it.
    date = datetime.strftime(date, '%d-%m-%Y')
    
    # Finally we save the graph in .graphml format.
    filename = f'graph_{date}.graphml'
    output_filepath = '/mnt/disk2/Data/Graphs_2/' + filename
    print(output_filepath)
    g.save(output_filepath)
    print(f"File '{filename}' successfully created and stored.")
    k += 1

In [8]:
# Load the graph files first. (24 minutes to load)
files = glob('/mnt/disk2/Data/Graphs/*.graphml')
    
graph_list = []  # List to store the loaded graphs

for i, file in tqdm(enumerate(files)):
    graph_name = file[file.find('graph_'):file.find('.graph')].replace('-','_')
    graph = nx.read_graphml(file)
    graph_list.append(graph)
    globals()[graph_name] = graph
    
del graph  
del graph_list

0it [00:00, ?it/s]

61it [21:54, 21.55s/it]


### STEPS TO LOAD DYNAMIC GRAPH IN GEPHI
##### 1) Open Gephi
##### 2) Import spreadsheet - nodes.csv
##### 3) Import timeset as TimeStamp
##### 4) Import spreadsheet - edges.csv
##### 5) Import timeset as TimeStamp